# Filtering Speed FIR vs FFT based on kernel size and image size

In [6]:
using LAP_julia, TimerOutputs, DataFrames, TableView, JLD2, FileIO, PyPlot, BenchmarkTools

In [7]:
filter_speed_df = DataFrame(
    index = Int[],
    filter_fun = Symbol[],
    img_size = Int[],
    kernel_half_size = Int[],
    timer = TimerOutput[])

kernel_half_sizes = cat(collect(1:51), collect(61:10:101), collect(141:40:501), collect(501:98:795), dims=1);
img_sizes = [100, 200, 400, 800, 1600, 3200];
filter_strat = []

In [ ]:
df = filter_speed_df
let index = 0
    for img_size in img_sizes

        # limit the max kernel size to 1/4 the image
        kernel_limit = img_size/4
        khs_modified = filter(x -> x <= kernel_limit ? x : 0, kernel_half_sizes)

        for khs in khs_modified
            
            for reg_fun in [sparse_lap_win_sum1, sparse_lap]
                if whs != 0
                    window = [whs * 2 + 1, whs * 2 + 1]
                    timer = TimerOutput("reg alg: sp lap")
                    # run once:
                    flow_est, source_reg, timer, results = test_registration_alg(reg_fun, img, imgw, flow, [whs, window], Dict(:timer => timer), timer=timer, display=false)
                    # add runs
                    if whs <= repeat_thresh
                        for _ in 1:repeat_count
                            flow_est, source_reg, timer, results = test_registration_alg(reg_fun, img, imgw, flow, [whs, window], Dict(:timer => timer), timer=timer, display=false)
                        end
                    end
                else
                    timer = TimerOutput("blank")
                    results = Dict()
                end

                index = index + 1
                println("at index: ", index, " img_size:", img_size, " whs: ", whs)
                push!(df, Dict(:index => index,
                               :reg_fun => Symbol(reg_fun),
                               :img_size => img_size,
                               :whs => whs,
                               :timer => timer,
                               :results => results))
            end
        end
    end
end